### NID Logistic

This is going to require a deeper understanding in order to convert this to binary classifier. Plus, there is simply not enough literature on this method.  

When you have time, dissect this and attempt to convert.  

Also, try to change to keras - something to blog about

This is on a subset (3000) and run with Comp premium as the response.   

For more info:

https://github.com/mtsang/neural-interaction-detection

Must run this in order. placeholders -> loss function -> session

In [2]:
import numpy as np
import pandas as pd
import math
import bisect
import operator
import tensorflow as tf
from sklearn.preprocessing import StandardScaler


In [22]:
data = pd.read_csv('../data/processed4/7_27_df_model.csv')
# skip all the synthetic stuff. 
data.drop(['Competitor.Premium'], axis=1, inplace=True)



In [23]:
#np.transpose(data.head())

In [24]:
data['TRV.Win'].value_counts()

0    18671
1     8028
Name: TRV.Win, dtype: int64

In [37]:
data.columns

Index(['TRV.Price', 'tot_bld_annl_rev_amt', 'fnl_prc_pt_fct', 'merge_freq',
       'w2v', 'program_freq', 'prcnt_blg_tiv', 'terr_pctl_bldgfire',
       'comp_freq', 'terr_pctl_liab', 'nbr_comp', 'terr_pctl_bldgcrime',
       'mnln_cd', 'tot_ref_rule_cnt', 'terr_pctl_weather', 'terr_pctl_water',
       'terr_pctl_cntscrime', 'rrv_rt_fct', 'terr_pctl_cntsfire',
       'SUM_BLD_SQFTAGE_bkt_medium', 'Segment_CONDOMINIUMS', 'Segment_OFFICE',
       'zip_freq', 'Region_CENTRAL', 'Segment_RESTAURANTS',
       'Segment_CONTRACTORS', 'State_CT', 'market_sh_low', 'Comp_rev_Hartford',
       'Region_WESTERN', 'State_TN', 'FirstQuoteYear_2016',
       'SUM_BLD_SQFTAGE_bkt_low', 'FirstEffYear_2018', 'UWC_KANSAS CITY',
       'State_VA', 'State_MI', 'Comp_rev_Other', 'Segment_APARTMENTS',
       'Segment_STORE', 'Region_NORTHEAST', 'earlyshop_mth_bkt_0',
       'SPRNKL_IND_bkt', 'market_sh_high', 'State_UT',
       'SUM_BLD_SQFTAGE_bkt_high', 'div_low', 'Segment_BUILDING', 'State_MS',
       'div_no

In [62]:
feat1 = np.asarray(data[['TRV.Price', 'tot_bld_annl_rev_amt', 'fnl_prc_pt_fct', 'merge_freq',
       'w2v', 'program_freq', 'prcnt_blg_tiv', 'terr_pctl_bldgfire',
       'comp_freq', 'terr_pctl_liab', 'nbr_comp', 'terr_pctl_bldgcrime',
       'mnln_cd', 'tot_ref_rule_cnt', 'terr_pctl_weather', 'terr_pctl_water',
       'terr_pctl_cntscrime', 'rrv_rt_fct', 'terr_pctl_cntsfire',
       'SUM_BLD_SQFTAGE_bkt_medium', 'Segment_CONDOMINIUMS', 'Segment_OFFICE',
       'zip_freq', 'Region_CENTRAL', 'Segment_RESTAURANTS',
       'Segment_CONTRACTORS', 'State_CT', 'market_sh_low', 'Comp_rev_Hartford',
       'Region_WESTERN', 'State_TN', 'FirstQuoteYear_2016',
       'SUM_BLD_SQFTAGE_bkt_low', 'FirstEffYear_2018', 'UWC_KANSAS CITY',
       'State_VA', 'State_MI', 'Comp_rev_Other', 'Segment_APARTMENTS',
       'Segment_STORE', 'Region_NORTHEAST', 'earlyshop_mth_bkt_0',
       'SPRNKL_IND_bkt', 'market_sh_high', 'State_UT',
       'SUM_BLD_SQFTAGE_bkt_high', 'div_low', 'Segment_BUILDING', 'State_MS',
       'div_none', 'UWC_CHARLOTTE', 'UWC_DALLAS', 'csf', 'market_sh_med',
       'bld_ind', 'UWC_HARTFORD', 'afi_pol_cnt',
       'program_rev_BUS PAC NAIL SALONS', 'LOC_BLD_AGE_bkt_highest',
       'program_rev_GARAGE PAC GENERAL REPAIRS WIT'
       ]].head(3000))
#feat1 = feat1.astype(float)

In [63]:
feat1.shape

(3000, 60)

In [64]:
y1 = np.asarray(data[['TRV.Win']].head(3000))
#y1 = y1.astype(float)



In [65]:
# scalery1 = StandardScaler()
# scalery1.fit(y1)
# y1 = scalery1.transform(y1)

### Train neural network

In [66]:
use_main_effect_nets = True # toggle this to use "main effect" nets


In [67]:
# Parameters
learning_rate = 0.01
num_epochs = 200
batch_size = 100
display_step = 100
l1_const = 5e-5
num_samples = 3000 #3k datapoints, split 1/3-1/3-1/3

# Network Parameters
n_hidden_1 = 140 # 1st layer number of neurons
n_hidden_2 = 100 # 2nd layer number of neurons
n_hidden_3 = 60 # 3rd "
n_hidden_4 = 20 # 4th "
n_hidden_uni = 10
num_input = 60 # simple synthetic example input dimension
num_output = 1 # regression or classification output dimension

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_output])

tf.set_random_seed(81)
np.random.seed(81)


In [68]:

tr_x1, va_x1, te_x1 = feat1[:1000], feat1[1000:2000], feat1[2000:]
tr_y1, va_y1, te_y1 = y1[:1000], y1[1000:2000], y1[2000:]


In [69]:
scaler_x = StandardScaler()
#scaler_y = StandardScaler()

scaler_x.fit(tr_x1)
#scaler_y.fit(tr_y1)

tr_x, va_x, te_x = scaler_x.transform(tr_x1), scaler_x.transform(va_x1), scaler_x.transform(te_x1)

#tr_y, va_y, te_y = scaler_y.transform(tr_y1), scaler_y.transform(va_y1), scaler_y.transform(te_y1)

tr_size = tr_x.shape[0]
print(tr_size)


1000


In [70]:
# access weights & biases
weights = {
    'h1': tf.Variable(tf.truncated_normal([num_input, n_hidden_1], 0, 0.1)),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], 0, 0.1)),
    'h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3], 0, 0.1)),
    'h4': tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_4], 0, 0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden_4, num_output], 0, 0.1))
}

biases = {
    'b1': tf.Variable(tf.truncated_normal([n_hidden_1], 0, 0.1)),
    'b2': tf.Variable(tf.truncated_normal([n_hidden_2], 0, 0.1)),
    'b3': tf.Variable(tf.truncated_normal([n_hidden_3], 0, 0.1)),
    'b4': tf.Variable(tf.truncated_normal([n_hidden_4], 0, 0.1)),
    'out': tf.Variable(tf.truncated_normal([num_output], 0, 0.1))
}

def get_weights_uninet():
    weights = {
        'h1': tf.Variable(tf.truncated_normal([1, n_hidden_uni], 0, 0.1)),
        'h2': tf.Variable(tf.truncated_normal([n_hidden_uni, n_hidden_uni], 0, 0.1)),
        'h3': tf.Variable(tf.truncated_normal([n_hidden_uni, n_hidden_uni], 0, 0.1)),
        'out': tf.Variable(tf.truncated_normal([n_hidden_uni, num_output], 0, 0.1))
    }
    return weights

def get_biases_uninet():
    biases = {
        'b1': tf.Variable(tf.truncated_normal([n_hidden_uni], 0, 0.1)),
        'b2': tf.Variable(tf.truncated_normal([n_hidden_uni], 0, 0.1)),
        'b3': tf.Variable(tf.truncated_normal([n_hidden_uni], 0, 0.1))
    }
    return biases


In [75]:
#Create model
def normal_neural_net(x, weights, biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
    layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['h3']), biases['b3']))
    layer_4 = tf.nn.relu(tf.add(tf.matmul(layer_3, weights['h4']), biases['b4']))    
    out_layer = tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits = tf.matmul(layer_4, weights['out']) + biases['out'])
    return out_layer

def main_effect_net(x, weights, biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
    layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['h3']), biases['b3']))    
    out_layer = tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=tf.matmul(layer_3, weights['out']))
    return out_layer

# L1 regularizer
def l1_norm(a): return tf.reduce_sum(tf.abs(a))


In [ ]:
# cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=h1)
# cost = tf.reduce_mean(cross_entropy)
# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [76]:
# Construct model
net = normal_neural_net(X, weights, biases)

if use_main_effect_nets:  
    me_nets = []
    for x_i in range(num_input):
        me_net = main_effect_net(tf.expand_dims(X[:,x_i],1), get_weights_uninet(), get_biases_uninet())
        me_nets.append(me_net)
    net = net + sum(me_nets)

# Define optimizer
loss_op = tf.losses.mean_squared_error(labels=Y, predictions=net)
# loss_op = tf.sigmoid_cross_entropy_with_logits(labels=Y,logits=net) # use this in the case of binary classification
sum_l1 = tf.reduce_sum([l1_norm(weights[k]) for k in weights])
loss_w_reg_op = loss_op + l1_const*sum_l1 

batch = tf.Variable(0)
decaying_learning_rate = tf.train.exponential_decay(learning_rate, batch*batch_size, tr_size, 0.95, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate=decaying_learning_rate).minimize(loss_w_reg_op, global_step=batch)


In [77]:
init = tf.global_variables_initializer()
n_batches = tr_size//batch_size
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.25
# config.gpu_options.allow_growth = True
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Initialized')

for epoch in range(num_epochs):

    batch_order = list(range(n_batches))
    np.random.shuffle(batch_order)

    for i in batch_order:
        batch_x = tr_x1[i*batch_size:(i+1)*batch_size]
        batch_y = tr_y1[i*batch_size:(i+1)*batch_size]
        _, lr = sess.run([optimizer,decaying_learning_rate], feed_dict={X:batch_x, Y:batch_y})

    if (epoch+1) % 50 == 0:
        tr_mse = sess.run(loss_op, feed_dict={X:tr_x1, Y:tr_y1})
        va_mse = sess.run(loss_op, feed_dict={X:va_x1, Y:va_y1})
        te_mse = sess.run(loss_op, feed_dict={X:te_x1, Y:te_y1})
        print('Epoch', epoch+1)
        print('\t','train rmse', math.sqrt(tr_mse), 'val rmse', math.sqrt(va_mse), 'test rmse', math.sqrt(te_mse))
        print('\t','learning rate', lr)
        
print('done')


Initialized
Epoch 50
	 train rmse 38.53897301961029 val rmse 44.292705381360484 test rmse 44.076965987509425
	 learning rate 0.00080994656
Epoch 100
	 train rmse 38.53549024975629 val rmse 44.54262601403921 test rmse 44.39480879395205
	 learning rate 6.232128e-05
Epoch 150
	 train rmse 38.53528276240003 val rmse 44.56607602017411 test rmse 44.42337399642853
	 learning rate 4.7953063e-06
Epoch 200
	 train rmse 38.53527009138025 val rmse 44.568438419188524 test rmse 44.426197358301444
	 learning rate 3.689745e-07
done


### Interprete weights

In [78]:
def preprocess_weights(w_dict):
    hidden_layers = [int(layer[1:]) for layer in w_dict.keys() if layer.startswith('h')]
    output_h = ['h' + str(x) for x in range(max(hidden_layers),1,-1)]
    w_agg = np.abs(w_dict['out'])
    w_h1 = np.abs(w_dict['h1'])

    for h in output_h:
        w_agg = np.matmul( np.abs(w_dict[h]), w_agg)

    return w_h1, w_agg 

def get_interaction_ranking(w_dict):
    xdim = w_dict['h1'].shape[0]
    w_h1, w_agg = preprocess_weights(w_dict)
        
    # rank interactions
    interaction_strengths = dict()

    for i in range(len(w_agg)):
        sorted_fweights = sorted(enumerate(w_h1[:,i]), key=lambda x:x[1], reverse = True)
        interaction_candidate = []
        weight_list = []       
        for j in range(len(w_h1)):
            bisect.insort(interaction_candidate, sorted_fweights[j][0]+1)
            weight_list.append(sorted_fweights[j][1])
            if len(interaction_candidate) == 1:
                continue
            interaction_tup = tuple(interaction_candidate)
            if interaction_tup not in interaction_strengths:
                interaction_strengths[interaction_tup] = 0
            inter_agg = min(weight_list)      
            interaction_strengths[interaction_tup] += np.abs(inter_agg*np.sum(w_agg[i]))
        
    interaction_sorted = sorted(interaction_strengths.items(), key=operator.itemgetter(1), reverse=True)

    # forward prune the ranking of redundant interactions
    interaction_ranking_pruned = []
    existing_largest = []
    for i, inter in enumerate(interaction_sorted):
        if len(interaction_ranking_pruned) > 20000: break
        skip = False
        indices_to_remove = set()
        for inter2_i, inter2 in enumerate(existing_largest):
            # if this is not the existing largest
            if set(inter[0]) < set(inter2[0]):
                skip = True
                break
            # if this is larger, then need to recall this index later to remove it from existing_largest
            if set(inter[0]) > set(inter2[0]):
                indices_to_remove.add(inter2_i)
        if skip:
            assert len(indices_to_remove) == 0
            continue
        prevlen = len(existing_largest)
        existing_largest[:] = [el for el_i, el in enumerate(existing_largest) if el_i not in indices_to_remove]
        existing_largest.append(inter)
        interaction_ranking_pruned.append((inter[0], inter[1]))

        curlen = len(existing_largest)

    return interaction_ranking_pruned

def get_pairwise_ranking(w_dict):
    xdim = w_dict['h1'].shape[0]
    w_h1, w_agg = preprocess_weights(w_dict)

    input_range = range(1,xdim+1)
    pairs = [(xa,yb) for xa in input_range for yb in input_range if xa != yb]
    for entry in pairs:
        if (entry[1], entry[0]) in pairs:
            pairs.remove((entry[1],entry[0]))

    pairwise_strengths = []
    for pair in pairs:
        a = pair[0]
        b = pair[1]
        wa = w_h1[a-1].reshape(w_h1[a-1].shape[0],1)
        wb = w_h1[b-1].reshape(w_h1[b-1].shape[0],1)
        wz = np.abs(np.minimum(wa , wb))*w_agg
        cab = np.sum(np.abs(wz))
        pairwise_strengths.append((pair, cab))
#     list(zip(pairs, pairwise_strengths))

    pairwise_ranking = sorted(pairwise_strengths,key=operator.itemgetter(1), reverse=True)

    return pairwise_ranking


In [79]:

w_dict = sess.run(weights)


In [80]:
# Variable-Order Interaction Ranking
get_interaction_ranking(w_dict)


[((32, 46), 22.210764408111572),
 ((6, 52), 21.3311505317688),
 ((33, 48), 20.884571075439453),
 ((24, 58), 19.49528408050537),
 ((11, 37), 19.436264038085938),
 ((40, 48), 17.930849075317383),
 ((33, 37, 48), 17.641664028167725),
 ((38, 54), 17.021543502807617),
 ((38, 54, 58), 16.592987060546875),
 ((38, 46, 54, 58), 16.38045310974121),
 ((48, 59), 16.26576805114746),
 ((46, 52), 16.123258590698242),
 ((39, 46, 52), 15.99014663696289),
 ((37, 60), 15.967248439788818),
 ((1, 21), 15.905470371246338),
 ((6, 37, 52), 15.815825939178467),
 ((26, 46), 14.92687702178955),
 ((36, 38, 46, 54, 58), 14.856783866882324),
 ((26, 33), 14.842087745666504),
 ((32, 40, 48), 14.354464530944824),
 ((37, 49), 14.228572845458984),
 ((36, 38, 44, 46, 54, 58), 13.86902141571045),
 ((35, 52), 13.795915603637695),
 ((4, 32, 40, 48), 13.659863471984863),
 ((35, 37), 13.631407022476196),
 ((43, 54), 13.131062984466553),
 ((22, 60), 13.015695571899414),
 ((4, 32, 38, 40, 48), 12.970699310302734),
 ((29, 39, 46

In [48]:
# Pairwise Interaction Ranking
get_pairwise_ranking(w_dict)


[((1, 61), 0.29975796),
 ((20, 61), 0.22080484),
 ((7, 61), 0.16251118),
 ((42, 61), 0.11667034),
 ((55, 61), 0.073094286),
 ((53, 61), 0.07162441),
 ((22, 61), 0.07133272),
 ((22, 42), 0.07133224),
 ((42, 53), 0.05191951),
 ((22, 53), 0.051919375),
 ((1, 7), 0.04215522),
 ((1, 20), 0.030900685),
 ((50, 61), 0.028677635),
 ((22, 50), 0.028677117),
 ((42, 50), 0.028677098),
 ((50, 53), 0.028677098),
 ((7, 20), 0.025947714),
 ((1, 42), 0.020934265),
 ((20, 53), 0.019705191),
 ((7, 53), 0.01970515),
 ((7, 55), 0.008823967),
 ((20, 42), 0.008045455),
 ((6, 61), 0.0073577217),
 ((6, 42), 0.0073569994),
 ((1, 6), 0.0073569333),
 ((6, 20), 0.0005725841),
 ((18, 61), 2.3829798e-06),
 ((40, 61), 2.2992374e-06),
 ((34, 61), 2.2869085e-06),
 ((46, 61), 2.2846366e-06),
 ((51, 61), 2.2829183e-06),
 ((56, 61), 2.2446925e-06),
 ((37, 61), 2.2351887e-06),
 ((8, 61), 2.207834e-06),
 ((25, 61), 2.143432e-06),
 ((39, 61), 2.1341048e-06),
 ((58, 61), 2.1293743e-06),
 ((16, 61), 2.121386e-06),
 ((48, 61), 